# Parse development book

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

The aggreated doc for dev: https://docs.google.com/document/d/1ga6R_9TWAQlXE9XqPE_qZ2S8S8aVW2A7n8SuYpSXnoI/edit#heading=h.q9ntf0ofam2u
The holy schema: https://app.diagrams.net/#G1bKvqEKr6JzPWryVg-vYudQy_4KvuzHS9

### JSon schemas for the actants
 * are created from *.ts files through typescript-jschema commandline utility
 * the schemas are trasformed as python Classes through warlock library


### Principles of the parsing operations
 * There are google sheet dataset, which need to be transformed to json format according to inkVisior datamodel
 * inkVisitor holds datamodel in the code, the typescript classes (see /shared/types...), or the holy schema
 * DZ created in input table parsing instructions
   * the first 5 rows contain instructions
   * if the keywords contain ? character, they are ignored (it is work in progress from DZ)

#### Parsing instructions
 * explicit set of keywords
   * discard
     * ...
   * inside
     * the value in the column should be straitforwadly made part of the entity object, the exists update_generic method or update_fieldName mehthods for fields with custom but generally applicable logic
   * propvalue
     * for making so called
          * statement or property statment (old A0093 has relation)
     * it sits in the entity "props" attribute, it has IProp class
   * special
     * fully custom method for the parsing behavior
     *

### Parsing process
 1. Prepare data model entities classes (from typescript inkvisitor classes -> json schema ->  python classes)
 2. Load and wrangle all input data
 3. Process headers of the tables for parsing instructions
 4. Process tables row by row, column by column
 5. Save as json file (which can be imported to the inkVisitor RethinkDB)


# Dev updates:  September
 - adding new users
 -


# Dev updates:  August
 - new instruction with proptype_2nd in column
    - before we got the type fixed and had just instruction propvalue_2nd (used in territories for manuscript witness)
 - new instruction "hook-inside" for insides in newly created entities (see in gugli locations special:modern_name)
 - changes :  some properties 'id' changed to 'specificId'
   - IProp {... 'value':{'entityId' and 'type':{'entityId'
   - IStatement {  'data':{ 'territory' {'territoryId'
   - ITerritory { 'data': {'parent': {'territoryId': ""
- bigger TODO relations

# Dev updates: April

## Concepts
  * DONE wordnet_synset_id
    * wordnet_id bounded to resource object R0067

## Texts
 * DONE special_creation_event_id
   * instructions:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
   * creation_event_label in standalone field
     * this will trigger the operation
   * lot of proptype and propvalue with non-standard schema: new branch of interpreting

## Manuscripts
 * special_repository_label
   * i:
     * For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 "class" - C2646 "archive or library", and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.
   * repository_label_language
     * i:
       * Use this value as label language value of the repository L entity.
 * DONE creation_event_id
   * creation_event_label
   * i:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
 * reproduction_online_url
   * instructions:
     * If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = "approved", label-language = "English", url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 "digital reproduction" - the R entity here generated.

# Unaccounted DEV Notes / instructions

## 21.09. 2022
* R table has received a new parsable column "components_id". Its notation follows our agreed-upon notation, so it should not pose any problem, but I rather signal this new col. to you. It is for composite Rs, which need to be separated into several identities with different URIs (typically two volumes of an edition)
* TODO The instruction here and here for generating creation events for Territories and manuscript Objects has changed, since what was the Property C0565 class is now the RelationType Classification. Please take this change into account, tx.
*



### Input variables

In [1]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  gdrive code, header_in_row, columns which need suffix based on a case name
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5), # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0,
    "guglielmites_persons" : ("Persons","1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk", 5, ['id','residence_id']), # GULIELMITES Persons, https://docs.google.com/spreadsheets/d/1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk/edit#gid=0,
    "guglielmites_locations" : ("Locations","1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs", 5, ['id','locationrelation1_value_id']) ,# GULIELMITES Locations, https://docs.google.com/spreadsheets/d/1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs/edit#gid=0,
    "guglielmites_events" : ("Events","1DtWsyOOR-QE4Zv1pPLQ6k4-gFmiS1Qkzi_zHnlNqAlU", 5, ['id','spatial_ref_value_id','participant_id','inquisitor_id','notary_id','witness_assessor_id','accuser_id','accused_id','mentioned_id','denied_id','marked_as_incriminated_id','temporal_ref_2_value_id']) # https://docs.google.com/spreadsheets/d/1DtWsyOOR-QE4Zv1pPLQ6k4-gFmiS1Qkzi_zHnlNqAlU/edit#gid=0
}

table_to_entity = {
    "concepts" : "IConcept",
    "resources" : "IResource",
    "texts" : "ITerritory",
    "manuscripts" : "IObject",
    "actions" :  "IAction",
    "guglielmites_persons" : "IPerson",
    "guglielmites_locations": "ILocation",
    "guglielmites_events": "IEvent",
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator
schema_path = '../schemas/' # path for dir with schemas
json_schemas = {}  # holder for schemas, so they can be used for jsonschema validate

In [2]:
import subprocess
subprocess.run("python generate-json-schemas.py", shell=True,capture_output=True)

CompletedProcess(args='python generate-json-schemas.py', returncode=0, stdout=b'typescript-json-schema ../../shared/types/relation.ts Relation.IModel --required --noExtraProps --defaultProps --id Relation.IModel --out ../schemas/Relation.IModel.schema.json\r\ntypescript-json-schema ../../shared/types/relation.ts Relation.IClassification --required --noExtraProps --defaultProps --id Relation.IClassification --out ../schemas/Relation.IClassification.schema.json\r\n', stderr=b'C:\\Users\\Tomas Hampejs\\AppData\\Roaming\\npm\\node_modules\\typescript-json-schema\\dist\\typescript-json-schema.js:1054\r\n            throw new Error("type ".concat(symbolName, " not found"));\r\n                  ^\r\n\r\nError: type Relation.IModel not found\r\n    at JsonSchemaGenerator.getSchemaForSymbol (C:\\Users\\Tomas Hampejs\\AppData\\Roaming\\npm\\node_modules\\typescript-json-schema\\dist\\typescript-json-schema.js:1054:19)\r\n    at generateSchema (C:\\Users\\Tomas Hampejs\\AppData\\Roaming\\npm\\no

### Libraries

In [11]:
import os, warlock, json
from datetime import datetime
import time
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.filterwarnings("ignore")

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid

def get_uuid_id():
    return str(uuid.uuid4())

def is_valid_uuid(val):
    try:
        uuid.UUID(str(val))
        return True
    except ValueError:
        return False


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [12]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2022-09-22 12:22:35 : Google authentification start
20 2022-09-22 12:22:35 : Google authentification end
20 2022-09-22 12:22:35 : Dator initiation succesfull end


In [32]:
# read all schemas inside and warlock them as globally available classes
schema_filenames = os.listdir(schema_path)
json_classes = {}
for schema in schema_filenames:
    name = schema.split(".schema")[0]
    file_handler = open(schema_path + schema,"r")
    schema_json = json.load(file_handler)
    json_schemas[name] = schema_json
    globals()[name] = warlock.model_factory(schema_json)
    json_classes[name] = globals()[name]
    logger.info("Class " + name + " available.")

logger.info(f"There are {len(json_classes.keys())} json classes available ({' '.join(json_classes.keys())}).")

2022-09-22 15:50:28,777 INFO Class IActant available.
2022-09-22 15:50:28,792 INFO Class IAction available.
2022-09-22 15:50:28,810 INFO Class IAudit available.
2022-09-22 15:50:28,826 INFO Class IConcept available.
2022-09-22 15:50:28,845 INFO Class IEntity available.
2022-09-22 15:50:28,860 INFO Class IEvent available.
2022-09-22 15:50:28,874 INFO Class ILabel available.
2022-09-22 15:50:28,888 INFO Class ILocation available.
2022-09-22 15:50:28,908 INFO Class IObject available.
2022-09-22 15:50:28,922 INFO Class IPerson available.
2022-09-22 15:50:28,935 INFO Class IProp available.
2022-09-22 15:50:28,949 INFO Class IReference available.
2022-09-22 15:50:28,966 INFO Class IResource available.
2022-09-22 15:50:28,982 INFO Class IStatement available.
2022-09-22 15:50:28,998 INFO Class ITerritory available.
2022-09-22 15:50:29,015 INFO Class IUser available.
2022-09-22 15:50:29,031 INFO Class IValue available.
2022-09-22 15:50:29,035 INFO Class Relation.IClassification available.
2022-

In [14]:
# factory for making entity objects, contains defaults with "prerequisities"
from datetime import datetime
import_note = "Import batch [development] " + str(datetime.now())

class InkVisitorJSONObjectFactory:

    classes = json_classes

    json_class_defaults = {
        'IAction':{
            'class':'A', 'id':'','legacyId':'', 'label':'', 'language':'', 'detail':'','data':{'entities':{'a1':[],'a2':[],'s':[]},'valencies':{'a1':'','a2':'','s':''}}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IConcept':{
            'class':'C', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IValue':{
            'class':'V', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IProp':{
            'bundleEnd':False,'bundleStart':False, 'certainty':'1', 'children':[], 'elvl':'1',  'id':'', 'logic':'1', 'mood':[], 'moodvariant':'1', 'bundleOperator':'a', 'type': {'entityId':'','elvl':'1','logic':'1','partitivity':'1','virtuality':'1'},'value':{'entityId':'','elvl':'1', 'logic':'1', 'partitivity':'1', 'virtuality':'1'}
        },
        'IResource':{
             'class':'R', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'url':'','partValueBaseURL':'','partValueLabel':''}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'Relation.IModel':{
            'id':'', 'type':'', 'entityIds': []
        },
        'Relation.IIdentification':{  # logic 1 = positive
            'id':'', 'type':'I', 'certainty':'1','entityIds': ['','']
        },
        'IObject':{
             'class':'O', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IStatement':{
             'class':'S', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'actants':[], 'actions':[], 'tags':[],'territory': {'territoryId':'','order':0}, 'text':''}, 'props':[], 'notes':[], 'status':'1','references':[]
        },
        'ITerritory':{
            'class':'T', 'id':'', 'legacyId':'','label':'', 'language':'', 'detail':'','data':{'parent':{ "territoryId": "T0", "order": 0 }}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'ILocation':{
            'class':'L', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IEvent':{
            'class':'E', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
        'IReference':{
           'id':'','resource':'','value':''
        },
        'IAudit':{
           'id':'','entityId':'','user':'','date':'','changes':{}
        },
        'IPerson':{
            'class':'P', 'id':'', 'label':'', 'language':'', 'detail':'','data':{'logicalType':'1'}, 'props':[], 'notes':[], 'status':'1', 'references':[]
        },
    }

    def __init__(self):
        for key, item in type(self).json_class_defaults.items():
            if 'notes' in item and  len(item['notes']) == 0:
                item['notes'].append(import_note)

    def make(self, entity_name, override_object=None):
        if override_object is None:
            override_object = {}
        object = type(self).json_class_defaults[entity_name]
        object.update(override_object)
        return type(self).classes[entity_name](deepcopy(object))

    def validate_defaults(self):
        for e in self.json_class_defaults:
            d.logger.info(f"Trying to validate class {e}.")
            test = self.make(e, self.json_class_defaults[e])
            d.logger.info(f"Class {e} validated.")


IOF = InkVisitorJSONObjectFactory()
IOF.validate_defaults()

2022-09-22 12:22:35,959 INFO Trying to validate class IAction.
2022-09-22 12:22:35,970 INFO Class IAction validated.
2022-09-22 12:22:35,972 INFO Trying to validate class IConcept.
2022-09-22 12:22:35,984 INFO Class IConcept validated.
2022-09-22 12:22:35,986 INFO Trying to validate class IValue.
2022-09-22 12:22:35,995 INFO Class IValue validated.
2022-09-22 12:22:35,997 INFO Trying to validate class IProp.
2022-09-22 12:22:36,006 INFO Class IProp validated.
2022-09-22 12:22:36,008 INFO Trying to validate class IResource.
2022-09-22 12:22:36,020 INFO Class IResource validated.
2022-09-22 12:22:36,022 INFO Trying to validate class Relation.IModel.
2022-09-22 12:22:36,027 INFO Class Relation.IModel validated.
2022-09-22 12:22:36,030 INFO Trying to validate class Relation.IIdentification.
2022-09-22 12:22:36,036 INFO Class Relation.IIdentification validated.
2022-09-22 12:22:36,038 INFO Trying to validate class IObject.
2022-09-22 12:22:36,052 INFO Class IObject validated.
2022-09-22 12:

# Load input datasets

In [15]:
# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values=None, unified_empty_value =''):
    if empty_values is None:
        empty_values = ["#N/A", "#VALUE!"] # 'NA', 'NS',
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value, regex=True)
    df. fillna(unified_empty_value, inplace=True)
    return df


def apply_suffix(string, suffix):
    if string in ['NS','NA']:
        return string
    mod_string = ""
    if "#" in string:
        strings = string.split("#")
        for s in strings:
            mod_string += " #"+ s.strip()+suffix
        mod_string = mod_string[2:]
    else:
        mod_string = string + suffix
    return mod_string


# load all input tables
tables = {}
header_infos = {}
entity_ids = {}
for key, sheet in input_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(sheet[0],root_sheet_url + sheet[1], sheet[0], fromCache=False, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True, numerize=False)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])

    # clean brackets
    tables[key] = tables[key].replace("\[","",  regex=True)
    tables[key] = tables[key].replace("\]","",  regex=True)


    # case study dataset legacyId needs a suffix
    if "_" in key:
        columns = sheet[3]
        suffix = "_" + key.split("_")[0]
        for c in columns:
            # tables[key][c] = tables[key][c] + suffix

            tables[key][c]  = tables[key][c].apply(lambda item: apply_suffix(item,suffix) if len(str(item)) > 0 else item)

            #tables[key].loc[~tables[key][c].isna(), "c"] = tables[key][c] + "_" + suffix
            # df.loc[df["gender"] == "male", "gender"] = 1
        logger.info(f"{key} For columns {c} the suffix {suffix} was glued.")

    # code for legacyId copy and uuid creation
    # tables[key]['legacyId'] = tables[key]['id'].copy()
    tables[key].insert(2, 'legacyId', tables[key]['id'].copy())

    # inform instructive header about the new column and what to do with it
    header_infos[key]['legacyId'] = ""
    header_infos[key].at[3,'legacyId'] = "inside"
    tables[key]['id'] = tables[key].apply(lambda x: get_uuid_id(), axis=1)  # generate unique id for each row
    # make id dictionaries (it is much faster to search for keys there in legacyId>id retrievals)
    ed = tables[key][["legacyId","id"]].set_index("legacyId")

    if "_" in key:   # legacyIds are case-specified, i.e. all locations can be together in one table
        key = key.split("_")[1]

    entity_ids[key] = ed["id"].to_dict()

2022-09-22 12:22:36,231 INFO Calling for texts with sheet_name Texts.


20 2022-09-22 12:22:37 : Loading dataset Texts
20 2022-09-22 12:22:37 : Opting for variant header at row 5.
20 2022-09-22 12:22:54 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2022-09-22 12:22:54 : Dropping empty columns in the dataset Texts : (1011, 115)
20 2022-09-22 12:22:55 : Deleted 860 empty rows by label.
20 2022-09-22 12:22:55 : Loaded and prepared dataset Texts : (151, 115)
20 2022-09-22 12:22:55 : Making pickle cache of  Texts with separeted header file : (151, 115)


2022-09-22 12:22:55,703 INFO Calling for manuscripts with sheet_name Manuscripts.


20 2022-09-22 12:22:56 : Loading dataset Manuscripts
20 2022-09-22 12:22:56 : Opting for variant header at row 5.
20 2022-09-22 12:22:58 : Hyperlinks were detected and transformed in columns ['reproduction_online_url', 'reproduction_note'].
20 2022-09-22 12:22:58 : Dropping empty columns in the dataset Manuscripts : (1036, 65)
20 2022-09-22 12:22:58 : Deleted 897 empty rows by label.
20 2022-09-22 12:22:58 : Loaded and prepared dataset Manuscripts : (139, 65)
20 2022-09-22 12:22:58 : Making pickle cache of  Manuscripts with separeted header file : (139, 65)


2022-09-22 12:22:58,865 INFO Calling for resources with sheet_name Resources.


...

20 2022-09-22 12:23:46 : Loading dataset Locations
20 2022-09-22 12:23:46 : Opting for variant header at row 5.


2022-09-22 12:23:50,269 INFO guglielmites_locations For columns locationrelation1_value_id the suffix _guglielmites was glued.
2022-09-22 12:23:50,276 INFO Calling for guglielmites_events with sheet_name Events.


20 2022-09-22 12:23:50 : Hyperlinks were detected and transformed in columns [].
20 2022-09-22 12:23:50 : Dropping empty columns in the dataset Locations : (998, 29)
20 2022-09-22 12:23:50 : Deleted 913 empty rows by label.
20 2022-09-22 12:23:50 : Loaded and prepared dataset Locations : (85, 29)
20 2022-09-22 12:23:50 : Making pickle cache of  Locations with separeted header file : (85, 29)
20 2022-09-22 12:23:51 : Loading dataset Events
20 2022-09-22 12:23:51 : Opting for variant header at row 5.


2022-09-22 12:24:05,560 INFO guglielmites_events For columns temporal_ref_2_value_id the suffix _guglielmites was glued.


20 2022-09-22 12:24:05 : Hyperlinks were detected and transformed in columns [].
20 2022-09-22 12:24:05 : Dropping empty columns in the dataset Events : (200, 72)
20 2022-09-22 12:24:05 : Deleted 71 empty rows by label.
20 2022-09-22 12:24:05 : Loaded and prepared dataset Events : (129, 72)
20 2022-09-22 12:24:05 : Making pickle cache of  Events with separeted header file : (129, 72)


In [16]:
# making empty table of values
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])
tables['relations'] = pd.DataFrame(columns=['id','type','logic','certainty','entityIds','origin'])

In [17]:
#tables["guglielmites_events"] = tables["guglielmites_events"].replace("\[","",  regex=True)
#tables["guglielmites_events"] = tables["guglielmites_events"].replace("\]","",  regex=True)

tables["guglielmites_events"]

,id,label,legacyId,label_language,entity_logical_type,resource_id,document_no,page_range_start,page_range_end,page_range_concatenated,...,marked_as_incriminated_label,negat_mentioned_in_margine,circumstance5_id,circumstance5_label,note,participant_sex,location_class,editor,done,checked
0,94e674ec-cb8d-4c8f-b0df-cf84a75b2e54,Abiuratio et obligatio Andree Saramite,E0001_guglielmites,Latin,definite,R0002,I.1,52,56,52-56,...,NA,,,,,m,camera officii inquisitionis,David Zbíral,y,
1,7421991c-398d-4a83-9c75-f6fee76778f1,Dictum suprascripti Andree,E0002_guglielmites,Latin,definite,R0002,I.2,56,60,56-60,...,NS,n,,,Acknowledges the guilt of Ricadona and Melior ...,m,camera officii inquisitionis,David Zbíral,y,
2,742cc215-411f-41d8-be01-d73bebc8a6c8,Precepta facta suprascripto Andree,E0003_guglielmites,Latin,definite,R0002,I.3,60,60,60,...,NA,,,,,m,camera officii inquisitionis,David Zbíral,y,y
3,7c79419b-5790-4ad8-b1d9-3d8c3adde01a,Abiuratio et obligatio Bellacare uxoris ser Bo...,E0004_guglielmites,Latin,definite,R0002,I.4,60,62,60-62,...,NA,,,,,f,ecclesia,David Zbíral,y,
4,cd145021-edfc-468f-b034-0612eaaeb25b,Dictum suprascripte Bellacare,E0005_guglielmites,Latin,definite,R0002,I.5,62,66,62-66,...,Mayfreda de Pirovano,y,C3434,investigator's marking as denial,Mentions her father Rugerius Demianus as a her...,f,ecclesia,David Zbíral,y,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,a2bbc992-bb77-48a8-bbeb-3cc7f563f9db,Dictum Iacobi de Ferno,E0125_guglielmites,Latin,definite,R0002,I.6,68,68,68,...,,,,,"This is cited as a plural: ""de quibus facta es...",m,,David Zbíral,,
125,f2802a9c-ac72-4adb-8b08-97b96633cbf7,Abiuratio Mayfrede de Pirovano in manibus Mayf...,E0126_guglielmites,Latin,definite,R0002,I.9,78,78,78,...,,,,,,f,ecclesia,David Zbíral,,
126,5b7c1f85-1ae1-462c-9bfd-e2bdbf163976,Abiuratio Girardi de Novaçano in manibus inqui...,E0127_guglielmites,Latin,definite,R0002,I.15,90,90,90,...,,,,,,m,,David Zbíral,,
127,b8456712-2b04-49a0-ab33-3b0514df1500,Dictum Andree Saramite scriptum per Mayfredum ...,E0128_guglielmites,Latin,definite,R0002,I.17,100,100,100,...,,,,,,m,,David Zbíral,,


In [18]:
entity_ids

{'texts': {'T1': 'c880ee10-5673-4c29-b749-8f69673cc036',
  'T2': 'ef94353b-ccb7-406c-b768-927965fff609',
  'T3': '1323cb19-558a-4e1b-99b1-a586f868f8fa',
  'T4': '0fc6f388-e2fb-401f-adca-267d3c6e8665',
  'T5': '2fa59b27-1851-4d48-98df-5b82b532a871',
  'T6': '4fdfc08a-c67d-4b46-a046-203689e1df2f',
  'T7': 'e9483246-c565-41d8-bd00-df046b40ddd3',
  'T8': '044a4bb7-91f2-4821-87f1-0f3520b532e6',
  'T9': 'a0bd36d2-de5c-4dc5-8e7f-747b445e3ede',
  'T10': 'ff55236a-5f6f-40dd-b196-8f671b882f68',
  'T11': '5f6553fe-7bbc-4add-9857-3115bb491284',
  'T12': 'a4045fd7-5c8c-466e-8ca6-1756929098c1',
  'T13': '5a77b0c9-1651-423e-889a-52c90a7aa96d',
  'T14': '6ebc6941-bfa0-41c7-8789-da355496099b',
  'T15': 'c32a2828-0f0d-4f2e-99fb-110071b19b57',
  'T16': '49a6b4a6-d732-4010-a183-755a89757c87',
  'T17': 'dc8ebe3a-74fc-4180-9a96-ebfd42d4bf4a',
  'T18': 'e6f2275d-993f-46f2-8771-ac02b32520c1',
  'T19': '9b61d823-c8cf-4ad6-ae93-fc1a6e839cab',
  'T20': '926cb279-ed66-4514-b2e9-7f52f939e330',
  'T21': '8d822f3a-e

In [19]:
tables['manuscripts'][tables['manuscripts']['reproduction_online_url']=="link"]

,id,label,legacyId,label_short,detail,label_language,class_id,class_label,component_id,component_label,...,reproduction_note,order_priority,order_date,delivery_date,price_czk,DEPRECATED_label,note,manuscript_history_note,editor,parsing_rows_explained


In [20]:
# manual corrections in the input data

### Responsible editors

#### Data
Only concepts have "editors" filled.

#### Specification

Only entities have audits, i.e. the prop object, the ref object don't. (Could be wrong?)

**Rationale:**  Each entity will have 2 audit creation records. One with the user "import", second with the user "responsible editor.

Implementaions possiblities:
  - make the audits during the parsing : the way
  - make the audits afer the parsing is done (for secondary created entities, I will not have the context, and will be problem to find an responsible editor
  -

In [22]:
tables['guglielmites_persons']['editor'].value_counts()

David Zbíral    223
Name: editor, dtype: int64

In [25]:
# the names in the input tables  vs their user ids
# the tuser are imported through the user.json file

# Jan Král
# Reima Välimäki
# Lidia Hinz-Wieczorek

editors = {"import":"0", "David Zbíral":"100", "Robert Shaw":"101", "Davor Salihović":"102", "Katia Riccardo":"103", "Jan Král":"104", "Reima Välimäki":"105","Lidia Hinz-Wieczorek":"106"}

# Declaration of controlling classes

In [29]:
additional_entities = []
relation_records = []
audits = []

# for controlling entity and mapping of its fields
class EntityMapper:

    # simple inside values mapping from input_values in gsheets to inkVisitor enums
    # field: { FROM  : TO }
    enum_mapper = {'language': {"English":"eng","Latin":"lat","Occitan":"oci","Middle English":"enm","Czech":"ces","Italian":"ita","French":"fra","German":"deu"},"status":{"approved":"1","pending":"0","discouraged":"2","warning":"3"}, "entity_logical_type":{'definite' : "1",
  'indefinite' : "2",   'hypothetical' : "3",  'generic' : "4"}}

    # status  Pending = "0",   Approved = "1",  Discouraged = "2",  Warning = "3",
    valid_entity_classes = ['A','C','E','O','R','T','P','G','S','L','NULL']


    IOF = InkVisitorJSONObjectFactory()

    def __init__(self, entity_type, data_row, logger = d.logger):
        self.entity =  type(self).IOF.make(entity_type)
        self.logger = logger
        self.debug = True
        self.data_row = data_row


    def make_prop_object(self,prop_type_id, prop_value_id):
        prop_object = IOF.make('IProp')
        prop_object['id'] = get_uuid_id()
        prop_object['type']['entityId'] = prop_type_id
        prop_object['value']['entityId'] = prop_value_id
        return prop_object

    def prepare_prop_object_data(self,prop_type, prop_value, origin = ""):
        prop_type_id = ""
        prop_value_id = ""

        # logger.info(f"Prepare prop object data>  {prop_type}, {prop_value}")

        # allowed entities in input
        allowed_strict_entities = ['C','M','A','E','L','R','S','V','O','T'] # should be followed by numbers
        allowed_free_string_entities = ['~V~']

        # checking input_value
        if not is_valid_uuid(prop_value):
            if any(prop_value.startswith(c)for c in allowed_strict_entities):
                # check for numbers
                if not prop_value[1:4].isnumeric():
                    prop_value = "~V~"+prop_value
            elif not any(prop_value.startswith(c)for c in allowed_free_string_entities):
                prop_value = "~V~"+prop_value

            # logger.info(f"Going to search for id for {prop_value}")
            prop_value_id = self.get_entity_id(prop_value, origin=origin)
        else:
           prop_value_id = prop_value

        # TODO this is useless now, because the line above will make ~V~ from everything unknown
        assert any(prop_value.startswith(c)  for c in allowed_strict_entities) or any(prop_value.startswith(c)  for c in allowed_free_string_entities) or is_valid_uuid(prop_value), f"Prop value unknown, C string or V string entity expected, or valid uuid.{prop_value}, {origin}"

        prop_type_id = self.get_entity_id(prop_type, origin=origin)

        return prop_type_id, prop_value_id



    def make_ref_object(self, ref_resource_id, ref_value_id):
        ref_object = IOF.make('IReference')
        ref_object['id'] = get_uuid_id()
        ref_object['resource'] = ref_resource_id
        ref_object['value'] = ref_value_id
        return ref_object

    def get_entity_id(self,entity_string, origin = ""):
        id = ""
        # logger.info(f"Getting entity string {entity_string} in {origin}")
        try:
            if entity_string.startswith("C") and entity_string[1:4].isnumeric():
                # id = tables['concepts'].loc[tables['concepts']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["concepts"][entity_string]
            elif entity_string.startswith("~V~"):
                ventity = self.make_ventity(entity_string, origin=origin)
                id = ventity['id']
            elif entity_string.startswith("M") and entity_string[1:4].isnumeric():
                # id = tables['manuscripts'].loc[tables['manuscripts']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["manuscripts"][entity_string]
            elif entity_string.startswith("A") and entity_string[1:4].isnumeric():
                # id = tables['actions'].loc[tables['actions']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["actions"][entity_string]
            elif entity_string.startswith("R") and entity_string[1:4].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["resources"][entity_string]
            elif entity_string.startswith("T") and entity_string[1:4].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["texts"][entity_string]
            elif entity_string.startswith("O") and entity_string[1:4].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["objects"][entity_string]
            elif entity_string.startswith("P") and entity_string[1:4].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["persons"][entity_string]
            elif entity_string.startswith("L") and entity_string[1:4].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["locations"][entity_string]
            elif entity_string.startswith("E") and entity_string[1:4].isnumeric():
                # id = tables['resources'].loc[tables['resources']['legacyId'] == entity_string,'id'].values[0]
                id = entity_ids["events"][entity_string]

            elif is_valid_uuid(entity_string):
                id = entity_string

        except IndexError as E:
            logger.info(f"Cannot get entity id from entity string {entity_string} in {origin}. {E}")

        if id != "" and isinstance(id, str):
            # logger.info(f"Got entity id {id} from entity string {entity_string} in {origin}")
            return id
        else:
            logger.error(f"Cannot get entity id from {entity_string}.")
            raise Exception(f"Cannot get entity id from {entity_string}.")

    def make_ventity(self, value_string, origin=""):
        # logger.info(f"Generating ventity from {value_string}.")
        # generate value entity object...
        ventity = IOF.make('IValue')
        ventity['id'] = get_uuid_id()
        ventity['label'] = value_string.replace("~V~","")

        if self.debug:
            ventity['notes'].append(origin)

        # register ventity
        tables['values'] = tables['values'].append({'id':ventity['id'] ,'value':ventity['label'],"origin":origin},ignore_index=True )
        additional_entities.append(ventity)

        # logger.info(f"Ventity id={ventity['id']} generated.")

        # create audit record
        self.create_audit_record(entity_id=ventity['id'], object=ventity)

        return ventity

    def make_rentity(self, label, url = "", origin=""):
        # logger.info(f"Generating rentity from {value_string}.")
        # generate resource entity object...
        rentity = IOF.make('IResource')
        rentity['id'] = get_uuid_id()
        rentity['label'] = label
        rentity['data']['url'] = url

        if self.debug:
            rentity['notes'].append(origin)

        # register rentity
        tables['resources'] = tables['resources'].append({'id':rentity['id'] ,'value':rentity['label'],"origin":origin}, ignore_index=True )
        additional_entities.append(rentity)

        # logger.info(f"Rentity id={rentity['id']} generated.")
        return rentity


    def make_eentity(self, label, legacyId, url = "", origin=""):

        eentity = IOF.make('IEvent')
        eentity['id'] = get_uuid_id()
        eentity['label'] = label
        eentity['legacyId'] = legacyId

        if self.debug:
            eentity['notes'].append(origin)

        # register event
        tables['events'] = tables['events'].append({'id':eentity['id'] ,'value':eentity['label'],"origin":origin, "legacyId":legacyId}, ignore_index=True )
        additional_entities.append(eentity)

        return eentity

    def make_lentity(self, label, legacyId="", url = "", origin=""):

        lentity = IOF.make('ILocation')
        lentity['id'] = get_uuid_id()
        lentity['label'] = label
        lentity['legacyId'] = legacyId

        if self.debug:
            lentity['notes'].append(origin)

        # register lentity
        tables['locations'] = tables['locations'].append({'id':lentity['id'] ,'value':lentity['label'],"origin":origin,"legacyId":legacyId}, ignore_index=True )
        additional_entities.append(lentity)

        return lentity


    def make_relation_identity_record(self, id1, id2, origin=""):

        rir = IOF.make('Relation.IIdentification')
        rir['id'] = get_uuid_id()
        rir['entityIds'] = [id1,id2]

        # register lentity
        tables['relations'] = tables['relations'].append({'id':rir['id'] ,'type':rir['type'], "certainty":rir['certainty'],"entityIds":rir['entityIds'], "origin":origin}, ignore_index=True )
        relation_records.append(rir)

        return rir

    def make_relation_classification_record(self, id1, id2, origin=""):

        rir = IOF.make('Relation.IClassification')
        rir['id'] = get_uuid_id()
        rir['entityIds'] = [id1,id2]

        # register lentity
        tables['relations'] = tables['relations'].append({'id':rir['id'] ,'type':rir['type'], "certainty":rir['certainty'],"entityIds":rir['entityIds'], "origin":origin}, ignore_index=True )
        relation_records.append(rir)

        return rir

    # interprets prop_type (should be always concept or resource) and input_value (should be concept or value string)
    # get ids of the prop_type and prop_value (possibly creates and register values object)
    # make iProp object
    # puts iProp object into the entity props property
    def hook_prop_object(self, prop_type, input_value, prop_source="",  origin="", prop_source_id ="", field_name="", prop_source_field = ""):

        # allowed entities in type
        assert "C" in prop_type[0] or "R" in prop_type[0], f"Prop type unknown, C or R string entity expected? {prop_type}, {origin}"

        prop_type_id, prop_value_id = self.prepare_prop_object_data(prop_type, input_value, origin=origin)
        # make IProp object
        prop_object = self.make_prop_object(prop_type_id, prop_value_id)

        # register prop object
        tables['props'] = tables['props'].append({'id':prop_object['id'] , 'type_id':prop_type_id,'value_id':prop_value_id,'type':prop_type,'value':input_value, "original_field":field_name, "origin":origin, 'entityId':self.entity['id'], 'legacyId':self.entity['legacyId']}, ignore_index=True)

        if prop_source_field != "":
            self.hook_2ndprop_into_props(prop_object,prop_source_field = prop_source_field)
        elif prop_source_id !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source_id = prop_source_id)
        elif prop_source !="": # means propvalue_2nd
            self.hook_2ndprop_into_props(prop_object,prop_source = prop_source)

        else:
            # hook directly into the entity object
            self.hook_prop_into_props(prop_object)

    def hook_ref_object(self, ref_legacyID, input_value, prop_source="",  origin=""):

        # allowed entities in ref_legacyID
        assert "R" in ref_legacyID, f"Unknown input, R legacyId expected? {ref_legacyID},{input_value}, {origin}"

        #modify value, so the value object is created
        input_value = "~V~"+input_value

        ref_resource_id = self.get_entity_id(ref_legacyID, origin=origin)
        ref_value_id = self.get_entity_id(input_value, origin=origin)

        # make IReference object
        ref_object = self.make_ref_object(ref_resource_id, ref_value_id)

        self.hook_ref_into_refs(ref_object)


    def hook_prop_into_props(self,prop_object):
        self.entity['props'].append(prop_object)

    def hook_ref_into_refs(self,ref_object):
        self.entity['references'].append(ref_object)

    def hook_2ndprop_into_props(self, prop_object, prop_source = "", prop_source_id = "", prop_source_field = ""):  # identification by concept id

        # recognition based on the prop_source_field
        if prop_source_field != "":

            # count, value in enumerate(values)
            for count, po in enumerate(self.entity['props']):
                # po['id']
                # find whether this id is registered under the prop_source_field within this entity frame
                candidate_prop_objects = tables['props'][(tables['props']['legacyId'] ==self.entity['legacyId']) & (tables['props']['original_field'] == prop_source_field)]

                logger.info(f"2nprop: {len(candidate_prop_objects)}")
                logger.info(f"2nprop: {len(candidate_prop_objects)} : {candidate_prop_objects.iloc[0]['id']}")

                if len(candidate_prop_objects) > 0:
                    for key, row in candidate_prop_objects.iterrows():

                        for  po in self.entity['props']:
                            if po['id'] == row['id'] and len(po['children']) == 0:
                                logger.info(f"Found prop. {po['id']}. Adding the 2ndprop child.")
                                po['children'].append(prop_object)

                else:
                    logger.error(f"Cannot find the proper prob object record. {prop_object} at {prop_source_field}")

        # recognition based on the type concept of the parent prop object
        if prop_source != "":
            keyId = self.get_entity_id(prop_source)
            assert is_valid_uuid(keyId), f"Cannot recognize entity id from {prop_source}"

            # count, value in enumerate(values)
            for count, po in enumerate(self.entity['props']):
                if po['type']['entityId'] == keyId and len(po['children']) == 0:  # I am counting on the fact, that if there are relations from multiples, they are processed in the specific right order
                   po['children'].append(prop_object)


        # recognition based od propobject id
        if prop_source_id != "":
            keyId = prop_source_id
            assert is_valid_uuid(keyId), "Not valid uuid {keyId}"

            for count, po in enumerate(self.entity['props']):
                if po['id'] == keyId:  # I am counting on the fact, that if there are relations from multiples, they are processed in the specific right order
                   po['children'].append(prop_object)



    # method invoker for the INSIDE operation with concrete fields
    def update_inside_field(self, field_name, input_value, origin= ""):
        if input_value != '':

            if ("#" in input_value or "~" in input_value) and field_name!= "note" and "https://docs." not in input_value:
                self.logger.info(f"ALERT # or ~ in the input value {input_value}")

            update_operation = "update_" + field_name
            # update_func = getattr(self, update_operation, self.update_generic)
            update_func = getattr(self, update_operation, self.update_generic)
            update_func(field_name, input_value, origin)
        else:
            raise Exception(f"Trying to update {field_name} with empty input value.")

    #########################################################################################################
    # the naming of procedures corresponds to the name of the input_table fields,  used for inside operations

    def update_label_language(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['language']:
            self.entity['language'] = type(self).enum_mapper['language'][input_value]
        else:
            self.logger.error(f"Unable to set language in {origin}.")
            self.entity['language'] = input_value # will raise error

    def update_status(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['status']:
            self.entity['status'] = type(self).enum_mapper['status'][input_value]
        else:
            self.logger.error(f"Unable to set status in {origin}.")
            self.entity['status'] = input_value # will raise error

    def update_entity_logical_type(self, field_name, input_value, origin = ""):
        if input_value in type(self).enum_mapper['entity_logical_type']:
            self.entity['data']['logicalType'] = type(self).enum_mapper['entity_logical_type'][input_value]
        else:
            self.logger.error(f"Unable to set entity logical type in {origin}.")
            self.entity['data']['logicalType'] = input_value # will raise error

    def update_note(self, field_name, input_value, origin = ""):
        #self.logger.info(f"Updating note with {input_value}.")
        if "#" in input_value:
            values = [v.strip() for v in input_value.split("#")]
            for v in values:
                self.entity['notes'].append(v)
        else:
            self.entity['notes'].append(input_value)

    def update_id(self, field_name, input_value, origin = ""):
        # self.entity['id'] = input_value
        self.entity['id'] = input_value

    def update_legacyId(self, field_name, input_value, origin = ""):
        # logger.info(f"Trying to set legacyId {type(input_value)}:'{input_value}' {origin}.")
        self.entity['legacyId'] = input_value

    def update_label(self, field_name, input_value, origin = ""):
        self.entity['label'] = input_value

    def update_wordnet_lemma_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_lemma_id NOT IMPLEMENTED ")
        pass

    def update_wordnet_synset_id(self, field_name, input_value, origin = ""):
        # self.logger.info(f" wordnet_synset_id NOT IMPLEMENTED ")
        pass

    def update_generic(self, field_name, input_value, origin = ""):
        self.entity[field_name] = input_value


    def create_audit_record(self, entity_id = '', editor_candidate = '', object = {}):

        data_row = self.data_row
        if entity_id == '':
            entity_id = self.entity['id']
        if object == {}:
            object = self.entity  # should be run at the end of the entity making, to the object is "full"

        # date =  {
        #     "$reql_type$": "TIME",
        #      "epoch_time": time.time(),
        #      "timezone": "+00:00"
        # }
        date =  datetime.now().strftime("%Y-%m-%dT%H:%M:%S")+"Z" #2021-12-05T19:10:15.739Z

        if editor_candidate == '' and 'editor' in data_row.keys():
            editor_candidate = data_row['editor']
        if editor_candidate in editors.keys():
            editor_id = editors[editor_candidate]
        else:
            editor_id = editors['David Zbíral']

        import_audit = IOF.make('IAudit')
        import_audit['id'] = get_uuid_id()
        import_audit['entityId'] = entity_id
        import_audit['user'] =  editors['import']
        import_audit['date'] = date
        import_audit['changes'] = object

        editor_audit = IOF.make('IAudit')
        editor_audit['id'] = get_uuid_id()
        editor_audit['entityId'] = entity_id
        editor_audit['user'] =  editor_id
        editor_audit['date'] = date
        editor_audit['changes'] = object

        # audits.append(import_audit)
        audits.append(editor_audit)


class TerritoryEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class ConceptEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class ActionEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger,):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_subject_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['s'].append(e)
            elif e == "*":
                self.entity['data']['entities']['s'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_subject_entity_type().")

    def update_subject_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_valency")
        self.entity['data']['valencies']['s'] = value

    def update_actant1_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a1'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a1'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant1_entity_type().")

    def update_actant2_entity_type(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant2_entity_type")
        entities = [e.strip() for e in value.split("|")]
        for e in entities:
            if e in self.valid_entity_classes:
                self.entity['data']['entities']['a2'].append(e)
            elif e == "*":
                self.entity['data']['entities']['a2'] = self.valid_entity_classes
            else:
                logger.error(f"Non valid entity processed: {e} while AP.update_actant2_entity_type().")

    def update_actant1_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a1'] = value

    def update_actant2_valency(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: actant1_valency")
        self.entity['data']['valencies']['a2'] = value


class ResourceEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

    def update_url(self, operation, value, entity_mapper):
        # self.logger.info(f"AP custom field: subject_entity_type")
        self.entity['data']['url'] = value


class ObjectEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class EventEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class LocationEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)

class PersonEntityMapper(EntityMapper):
    def __init__(self,entity_type, data_row, logger = d.logger):
        EntityMapper.__init__(self,entity_type, data_row, logger)


class EntityMapperFactory:
    def __init__(self):
        pass

    def make(self, entity_name, data_row):
        if 'ITerritory' == entity_name:
            return TerritoryEntityMapper(entity_name, data_row)
        elif 'IAction' == entity_name:
            return ActionEntityMapper(entity_name, data_row)
        elif 'IConcept' == entity_name:
            return ConceptEntityMapper(entity_name,data_row)
        elif 'IResource' == entity_name:
            return ResourceEntityMapper(entity_name, data_row)
        elif 'IObject' == entity_name:
            return ObjectEntityMapper(entity_name, data_row)
        elif 'IEvent' == entity_name:
            return EventEntityMapper(entity_name, data_row)
        elif 'ILocation' == entity_name:
            return LocationEntityMapper(entity_name, data_row)
        elif 'IPerson' == entity_name:
            return LocationEntityMapper(entity_name, data_row)

        else:
            logger.warning(f"Unrecognized entity class in entity mapper. Is this right?")
            return EntityMapper(entity_name)





# CONTROL CLASS
class ParseController():

    def __init__(self, entity_list: [], keyword_row_id = 3,  logger = d.logger):
        self.entity_list = entity_list
        self.logger = logger
        self.parsers = {}
        self.js_objects = []

        for e in self.entity_list:
            if 'texts' in e:
                self.parsers[e] = TextParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'actions' in e:
                self.parsers[e] = ActionParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'concepts' in e:
                self.parsers[e] = ConceptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'resources' in e:
                self.parsers[e] = ResourceParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'manuscripts' in e:
                self.parsers[e] = ManuscriptParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'guglielmites_persons' in e:
                self.parsers[e] = PersonParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'guglielmites_locations' in e:
                self.parsers[e] = LocationParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            elif 'guglielmites_events' in e:
                self.parsers[e] = EventParser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)
            else:
                self.logger.warning(f"Coming to basic Parser entity - strange '{e}' {type(e)}.")
                self.parsers[e] = Parser(e, header_df = header_infos[e], table_df = tables[e], keyword_row_id = keyword_row_id, logger = logger)

    def load_json_objects(self):
        for name, p in self.parsers.items():
            self.js_objects = self.js_objects + p.js_objects

    def parse(self, stop = False):
        for name, p in self.parsers.items():
            p.parse_rows(stop)


# WORKER CLASS
class Parser():
    EMP = EntityMapperFactory()

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        self.name = name
        self.logname = name.upper()
        self.input_header_df = header_df
        self.input_table_df = table_df
        self.prepared_table = pd.DataFrame()
        self.keyword_row_id =  keyword_row_id
        self.columns = self.input_header_df.columns.tolist()

        self.proptype_2nd = {} # for registering the columns, which containt proptype_2nd information

        self.parsing_instruction = {}
        self.oper_columns = {'discard':[],'inside':[],'special':[],'unknown':[],"proptype":[],'propvalue':[],'proptype_2nd':[],'propvalue_2nd':[],"dependent":[], "reference_object":[], "reference_part":[],"hooked-inside":[],"hooked-propvalue":[]}
        self.logger = logger

        # parsed json data holder
        self.js_objects = []

        # RUN
        self.process_header_instructions()
        self.prepare_input_table()

    # "parsing" instructions
    def process_header_instructions(self) -> (pd.DataFrame, pd.DataFrame):
        keyword_row = self.input_header_df.iloc[self.keyword_row_id]
        prop_type_row = self.input_header_df.iloc[self.keyword_row_id - 1]
        source_node_row = self.input_header_df.iloc[self.keyword_row_id - 2]

        log_uncertain_instructions = []

        for c in self.columns:
            instruction_candidate = str(keyword_row.at[c]).strip()
            prop_type_candidate = str(prop_type_row.at[c]).strip()
            source_node_candidate = str(source_node_row.at[c]).strip()

            if c == '':
                self.logger.error(f"{self.logname} There is empty column in the dataset.")
                raise Exception(f"{self.logname} There is empty column in the dataset.")

            if "?" in instruction_candidate or "?" in prop_type_candidate or "?" in source_node_candidate:
                log_uncertain_instructions.append(f"{c.upper()}:{instruction_candidate},{prop_type_candidate},{source_node_candidate}")
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)


            # known instructions
            if 'discard' in instruction_candidate:
                instruction  = {'operation':'discard', 'target': None}
                self.oper_columns['discard'].append(c)

            elif instruction_candidate == "propvalue":
                prop_type = prop_type_candidate
                source_node = source_node_candidate

                # test whether is its propvalue proper or dependent (=proptype is dynamic, value from another column)
                if prop_type.strip() == "":
                    self.oper_columns['dependent'].append(c)
                    continue # ignoring "dependent propvalue"

                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue', 'type': prop_type, 'source':source_node} # source can be ignored, because the iProp object is sitting inside of it
                    self.oper_columns['propvalue'].append(c)

            elif 'propvalue_2nd' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'propvalue_2nd', 'type': prop_type, 'source':source_node} # source can NOT be ignored, it signals which existing iProp object will hold this iProp object
                    self.oper_columns['propvalue_2nd'].append(c)

            elif 'proptype_2nd' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                if "?" in prop_type or "?"  in source_node:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'proptype_2nd', 'type': prop_type, 'source':source_node}
                    self.oper_columns['proptype_2nd'].append(c)

            elif 'special' in instruction_candidate:
                # looks for custom functions registered by column name
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'special', 'type': prop_type, 'source':source_node}
                self.oper_columns['special'].append(c)

            elif 'proptype' in instruction_candidate:
                prop_type = prop_type_candidate
                source_node = source_node_candidate
                instruction  = {'operation':'proptype', 'type': prop_type, 'source':source_node}
                #logger.info(f"here ...{instruction_candidate} {c}")
                self.oper_columns['proptype'].append(c)

            elif 'dependent' in instruction_candidate:
                # ignore
                # the value is solved by another instruction
                instruction  = {'operation':'dependent', 'type': prop_type, 'source':source_node}

            elif instruction_candidate == "inside":
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'inside', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'inside', 'target': prop_type_candidate}

                    self.oper_columns['inside'].append(c)

            elif instruction_candidate == "hooked-inside":
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'hooked-inside', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'hooked-inside', 'target': prop_type_candidate}

                    self.oper_columns['hooked-inside'].append(c)

            elif instruction_candidate == "hooked-propvalue":
                if "?" in c:
                    instruction = {'operation':'unknown', 'target': None}
                    self.oper_columns['unknown'].append(c)
                else:
                    instruction  = {'operation':'hooked-propvalue', 'target': None}
                    if len(prop_type_candidate) > 0:
                        instruction  = {'operation':'hooked-propvalue', 'target': prop_type_candidate}

                    self.oper_columns['hooked-propvalue'].append(c)

            elif "reference_object" in instruction_candidate:
                #logger.info(f"Found instruction 'reference'. {prop_type_candidate}")
                self.oper_columns["reference_object"].append(c)
                instruction  = {'operation':'reference_object', 'ref_legecy_id':prop_type_candidate}

            elif "reference_part" in instruction_candidate:
                #logger.info(f"Found instruction 'reference'. {prop_type_candidate}")
                self.oper_columns["reference_part"].append(c)
                instruction  = {'operation':'reference_part', 'ref_legecy_id':prop_type_candidate}

            else:
                instruction = {'operation':'unknown', 'target': None}
                self.oper_columns['unknown'].append(c)
            self.parsing_instruction[c] = instruction

        self.logger.info(f"{self.logname} Uncertain parsing instructions in {len(log_uncertain_instructions)} columns: " + " ".join(log_uncertain_instructions) + ".")
        return self.parsing_instruction

    def prepare_input_table(self):
        ip = self.input_table_df.copy()

        # discard  columns with discard and unknown operations
        ip.drop(columns=self.oper_columns['discard']+self.oper_columns['unknown'], inplace=True)

        self.logger.info(f"{self.logname} {len(self.oper_columns['discard']+self.oper_columns['unknown'])} columns have been dropped (discard:{len(self.oper_columns['discard'])}, unknown:{len(self.oper_columns['unknown'])}: {self.oper_columns['unknown']}). Table now has {len(ip.columns)} columns, inside:{len(self.oper_columns['inside'])},propvalue:{len(self.oper_columns['propvalue'])}, special:{len(self.oper_columns['special'])}, proptype: {len(self.oper_columns['proptype'])}, proptype_2nd: {len(self.oper_columns['proptype_2nd'])}, dependent:{len(self.oper_columns['dependent'])}, reference_part:{len(self.oper_columns['reference_part'])}. Originally {self.input_table_df.shape[1]} columns.")

        self.prepared_table = ip

    def prepare_property(self):
        pass

    def make_row_object(self, data_row):
        class_name = table_to_entity[self.name]
        return self.EMP.make(class_name, data_row)

    def itemize_valuestring_for_multiples(self, value_with_multiples, origin="") -> []:
        values = []
        if isinstance(value_with_multiples,str):
            parsed_value = value_with_multiples.split('#')
            values =  [item.strip() for item in parsed_value]
        else:
            raise Exception(f"Expected value to be string. Got {type(value_with_multiples)}. {origin}")
        return values

    def parse_rows(self, stop = False):
        self.logger.info(f"Starting to parse {self.name}.")

        if  self.prepared_table['label'].isnull().any():
            self.prepared_table  = self.prepared_table[self.prepared_table['label'].notna()]
            self.logger.info(f"Empty labels found in {self.name} table. (new entities added through parsing process). Adjusting - entities with empty labels not taken into account.")


        counter = 0
        for key, row in self.prepared_table.iterrows():
            counter += 1
            if stop and counter > stop:
                break

            # self.logger.info(f"{self.name} Processing row {key}")
            entity_mapper = self.make_row_object(row.to_dict())

            for name, value in row.items():
                if name in self.parsing_instruction:
                    operation = self.parsing_instruction[name]
                else:
                    continue # silently ignore unknown columns
                # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                # force string
                value = str(value)

                if operation['operation'] == 'inside' and value != '' and '?' not in name:
                    # logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    if operation['target']:
                        name = operation['target']
                    entity_mapper.update_inside_field(name,value,operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value))

                if operation['operation'] == 'propvalue' and value != '':

                    logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']

                    if prop_type in row.keys():  # prop type is defined in other column (it is NOT fixed for all propvalues)
                        prop_type = row[prop_type]

                    if prop_type == '' or 'C' not in prop_type:
                        raise Exception(f"Propvalue does not have prop type '{prop_type}' defined. C entity-string expected, got {key}, {name}, {value}. [{operation}]")
                    for item in self.itemize_valuestring_for_multiples(value):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value), field_name = name)


                if operation['operation'] == 'proptype_2nd' and value != '':
                    logger.info(f"{self.name} Processing columns {name}, with value {value}. Op:{operation}")

                    #register proptype_2nd for the source
                    self.proptype_2nd[operation['source']] = name

                    # prop_type = operation['type']
                    # if prop_type == '' or 'C' not in prop_type:
                    #     raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got {key}, {name}, {value}")
                    # for item in self.itemize_valuestring_for_multiples(value):
                    #     entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value), field_name = name)

                if operation['operation'] == 'propvalue_2nd' and value != '':
                    #logger.info(f"{self.name} propvalue_2nd : Processing columns {name}, with value {value}. Op:{operation}")
                    prop_type = operation['type']
                    prop_source_name = operation['source']  # header_name,  we need concept_id
                    prop_source_id = self.parsing_instruction[prop_source_name]['type']

                    logger.info(f"{prop_source_name} : {prop_source_id} : {value}")

                    assert 'C' in prop_source_id, f"Trying to get to the concept of 1st level property to adress 2nd level property."

                    # there can be two situations
                    # B - proptype is defined in proptype_2nd column
                    if prop_source_name in self.proptype_2nd.keys():
                        logger.info(f"We have registered source type column {self.proptype_2nd[prop_source_name]} for this propvalue_2nd.")
                        prop_type = row[self.proptype_2nd[prop_source_name]]
                    else:
                        # A - proptype is fixed for the whole column
                        if prop_type == '' or 'C' not in prop_type:
                            raise Exception(f"Propvalue does not have prop type defined. C entity-string expected, got prop_type '{prop_type}', key:{key}, name:{name}, value:{value}")

                    origin = operation['operation'] +">"+ str(key)+":"+str(name)+":"+str(value)

                    for item in self.itemize_valuestring_for_multiples(value, origin=origin):
                        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = item, prop_source = prop_source_id, origin = origin, field_name = name, prop_source_field = prop_source_name)

                if operation['operation'] == 'special' and value != "":
                    # logger.info(f"SPECIAL {operation} {value}")
                    func = getattr(self, 'special_'+name)
                    func(operation, value, entity_mapper)

                if operation['operation'] == 'reference_object' and value != "":
                    pass

                if operation['operation'] == 'reference_part' and value != "":
                    ref_legecy_id = operation['ref_legecy_id']

                    if ref_legecy_id in row.keys():
                        ref_legecy_id = row[ref_legecy_id]

                    # logger.info(f"Doing {operation}, {ref_legecy_id}, '{value}'")
                    entity_mapper.hook_ref_object(ref_legacyID = ref_legecy_id, input_value = value, origin = operation['operation'] +">"+ ":"+ name + " " +str(value))

            self.js_objects.append(entity_mapper.entity)

            # make audit records
            entity_mapper.create_audit_record()

            # break  DEV, checking parsing after first iteration



    def special_editor(self, operation, value, entity_mapper, field_name="special_editor"):
        pass


class TextParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    # special methods for fields, which needs fully individual processing
    def special_edition_1(self, operation, value, entity_mapper, field_name="edition_1", ):
        # Parse this col. as "propvalue" - but you need to generate the target entities since they do not exist. How to do it: for any value, create an R entity with "label" = textual value in this col., "label language" = English, "status" = "approved", and "URL" = the hyperlink in the formula sitting on the textual value in this col. As usual, ignore NS and NA values (exact match) - do not import anything if the value is NA.
        # logger.info(f"Special edition1 running ...{operation} {value}")

        origin = operation['operation'] +" "+ operation['type']+ ">"+ ":"+field_name + str(value)
        prop_type = operation['type']

        # make rentity
        # logger.info(f"Rentity making {value}")
        if "|" in value:
            data = value.split("|")
            label = data[0]
            url = data[1]
        else:
            url = ""
            label = value
            # logger.warning(f"Expected char | signaling url after label. Got just {value}."+origin)

        rentity = entity_mapper.make_rentity(label, url, origin=origin)
        rentity['language'] = entity_mapper.enum_mapper['language']['English']
        entity_mapper.create_audit_record(entity_id=rentity['id'], object=rentity)

        entity_mapper.hook_prop_object(prop_type = prop_type, input_value = rentity['id'], origin = origin, field_name=field_name)

    def special_edition_2(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_2")

    def special_edition_3(self, operation, value, entity_mapper):
        self.special_edition_1( operation, value, entity_mapper, field_name="edition_3")

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).


        # new instructionds
        # TODO
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to this E the Relation of the type Classification leading to C2642 "creation".

        data = entity_mapper.data_row
        origin = operation['operation']+ " " + data['legacyId'] + ">"+ ":"+field_name +" "+ str(value)

        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value, origin = origin)
        event_entity['language'] = entity_mapper.enum_mapper['language']['English']

        prop_type_id = entity_mapper.get_entity_id("C0565")
        prop_value_id = entity_mapper.get_entity_id("C2642")

        # make IProp object
        prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)

        # hook prop object to the event entity
        event_entity['props'].append(prop_object)

        # hook the vent event to the territory
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = origin, field_name=field_name)

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]


        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)

        # create audit record
        entity_mapper.create_audit_record(entity_id=event_entity['id'], object=event_entity)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass

class ActionParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)


class ConceptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_wordnet_synset_id(self, operation, value, entity_mapper,field_name="special_wordnet_synset_id",):
        # wordnet_resource_id = R0067
        entity_mapper.hook_ref_object(ref_legacyID = "R0067", input_value = value, origin = operation['operation'] +">"+ ":"+field_name + " " +str(value))


class ManuscriptParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_creation_event_id(self, operation, value, entity_mapper : EntityMapper, field_name="creation_event_id"):
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new “hash” ID from the db, (3) label of this E: see next col., (4) logical type “definite” (default), (5) label language “English”, (6) status “approved”, and (7) attach to any of those Es the metaprop "(has) - C0565 “class” - C2642 “creation” (to instantiate the event to its event type = event class).

        # new instructions
        # TODO
        # Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: take from next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to this E the Relation of the type Classification leading to C2642 "creation".

        origin = operation['operation'] +">"+ ":"+field_name +" "+ str(value)

        data = entity_mapper.data_row
        event_entity = entity_mapper.make_eentity(data['creation_event_label'], legacyId = value, origin = origin)
        event_entity['language'] = entity_mapper.enum_mapper['language']['English']

        prop_type_id = entity_mapper.get_entity_id("C0565")
        prop_value_id = entity_mapper.get_entity_id("C2642")

        # make IProp object
        prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)


        # hook prop object to the event entity
        event_entity['props'].append(prop_object)

        # hook the vent event to the territory
        entity_mapper.hook_prop_object(prop_type = "C2642", input_value = event_entity['id'], origin = operation['operation'] +">"+ ":"+field_name + str(value))

        # process time_relations
        t_relations = [(
            data["timerelation1_type_conceptified_id"], data["timerelation1_target_id"]
        ), (
            data["timerelation2_type_conceptified_id"], data["timerelation2_target_id"]
        ),(
            data["timerelation3_type_conceptified_id"], data["timerelation3_target_id"]
        ), (
            data["timerelation4_type_conceptified_id"], data["timerelation4_target_id"]
        )]

        origin = origin + "LegacyId: "+ data['legacyId']
        for o in t_relations:
            if len(o[0]) > 0 and len(o[1]) > 0:
                prop_type_id = entity_mapper.get_entity_id(o[0], origin = origin)
                prop_value_id = entity_mapper.get_entity_id("~V~"+o[1], origin = origin)
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                event_entity['props'].append(prop_object)

        # create audit record
        entity_mapper.create_audit_record(entity_id=event_entity['id'], object=event_entity)


    # empty, operation is solved by f above
    def special_creation_event_label(self, operation, value, entity_mapper):
        pass


    def special_repository_label(self, operation, value, entity_mapper, field_name = "repository_label"):
        # For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = “approved”, entity logical type = “definite”, label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 “class” - C2646 “archive or library”, and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 “repository” - L in this col.

        origin = f"Making location '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value
            lentity = entity_mapper.make_lentity(label=value, legacyId="L_from_"+entity_mapper.data_row['legacyId'], origin=origin)

            lentity['language'] = entity_mapper.enum_mapper['language'][entity_mapper.data_row['repository_label_language']]

            prop_type_id = entity_mapper.get_entity_id("C0565", origin = origin)
            prop_value_id = entity_mapper.get_entity_id("C2646", origin = origin)
            prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
            lentity['props'].append(prop_object)

            # let the manuscript O own the location
            entity_mapper.hook_prop_object(prop_type = "C2645", input_value = lentity['id'], origin = origin)

            # create audit record
            entity_mapper.create_audit_record(entity_id=lentity['id'], object=lentity)


    # solves the method above
    def special_repository_label_language(self, operation, value, entity_mapper):
        pass

    def special_reproduction_online_url(self, operation, value, entity_mapper, field_name = "reproduction_online_url"):
        # If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = “approved”, label-language = “English”, url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 “digital reproduction” - the R entity here generated.

        origin = f"Making Resource entity '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value

            if "|" in value:
                data = value.split("|")
                label = data[0]
                url = data[1]

                if label == "link":
                    label = "Reproduction of " + entity_mapper.data_row['label']

            else:
                url = ""
                label = value
                logger.warning(f"Expected char | signaling url after label. Got just {value}. "+origin)

            rentity = entity_mapper.make_rentity(label, url, origin=origin)
            rentity['language'] = entity_mapper.enum_mapper['language']['English']
            entity_mapper.hook_prop_object(prop_type = "C1199", input_value = rentity['id'], origin = origin)

            # create audit record
            entity_mapper.create_audit_record(entity_id=rentity['id'], object=rentity)


class ResourceParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

class PersonParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

class EventParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)


    def special_subterritory_id(self, operation, value, entity_mapper, field_name = "subterritory_id"):
        # Light yellow: document aka Territory metadata (not Event).
        # please generate these T entities following Adam's script for parsing Robert's Sellan coding sheet IDs into a hierarchical T structure
        # and put them under their proper root T through legacy IDs (first element of ID = root T ID).
        # E.g. here, in Guglielmites, there is root, i.e. the whole T; then parts 1-4; and underneath them, the individual documents.
        # Entity type = T; T label and label_language defined in the two respective cols.; entity_logical_type = definite; status = approved. Other metadata set as metaprops in the following yellow cols.
        pass


class LocationParser(Parser):

    def __init__(self, name, header_df: pd.DataFrame, table_df: pd.DataFrame, keyword_row_id: int, logger: logger):
        Parser.__init__(self, name, header_df, table_df, keyword_row_id, logger)

    def special_modern_name(self, operation, value, entity_mapper, field_name = "special_modern_name"):
        # "For non-empty, generate new L entities from this col.
        # 1) Attribute values: label = this col.; status = ""approved"", label_language = next col., entity_logical_type = same as col. D.
        # 2) Create ""identical entity"" relation between the Latin entity captured by this row (""id"") and this new entity.  Exact structure TBA (awaiting Adam) - until it is, create the relation as follows: append a metaprop to the original Latin location in ""id"", which will say: [""id"" Location] - has - C0691 identical entity - [modern location created from modern_name].)
        # 3) Attach ""class"" metaprop to this modern location from col. ""modern_name_class_id"" (described by the regular instructions)."

        origin = f"Making Location entity '{value}' from " + entity_mapper.data_row['legacyId'] + f" by field {field_name}."
        language = entity_mapper.data_row['modern_name_language']

        if value != "" and value!="NA" and value!="N/A" and value!="NS":  # check value
            label = value
            lentity = entity_mapper.make_lentity(label, origin=origin)
            lentity['language'] = entity_mapper.enum_mapper['language'][language]


            # create settlement prop
            #  pt C0565  value in modern_name_class_id
            prop_type = "C0565"
            prop_value = entity_mapper.data_row['modern_name_class_id'] # another C

            for item in self.itemize_valuestring_for_multiples(prop_value, origin=origin):
                prop_type_id, prop_value_id = entity_mapper.prepare_prop_object_data(prop_type, item, origin=origin)

                # make IProp object
                prop_object = entity_mapper.make_prop_object(prop_type_id, prop_value_id)
                # append prop to the location entity
                lentity['props'].append(prop_object)

            # make identity relation
            # {type: RelationType.Identification, entityIds: [Location 1 id, Location 2 id], certainty: [Certainty.Certain]},
            # {type: "I", entityIds: [Location 1 id, Location 2 id], certainty: "1"},
            entity_mapper.make_relation_identity_record(entity_mapper.entity['id'], lentity['id'])

            # create audit record
            entity_mapper.create_audit_record(entity_id=lentity['id'], object=lentity)



In [27]:
tables

{'texts':                                        id  \
 0    c880ee10-5673-4c29-b749-8f69673cc036   
 1    ef94353b-ccb7-406c-b768-927965fff609   
 2    1323cb19-558a-4e1b-99b1-a586f868f8fa   
 3    0fc6f388-e2fb-401f-adca-267d3c6e8665   
 4    2fa59b27-1851-4d48-98df-5b82b532a871   
 ..                                    ...   
 146  3d9aabc0-6dc9-442f-ab72-1a0eabcbcf15   
 147  711f6bbd-73e7-461f-933e-c189055dfe0f   
 148  5e4c198a-6507-423f-a294-74d0a9c78989   
 149  1d648aff-b179-4b5f-b57f-6457f73e5145   
 150  01604edc-9500-4955-9006-767030da45bc   
 
                                                  label legacyId  \
 0         Process against Bernard Niort and his family       T1   
 1    Sentences of William Arnold and Stephen of Sai...       T2   
 2                   Peter Seila’s Register of Penances       T3   
 3          Register FFF of the Carcassonne inquisition       T4   
 4    Confirmation of depositions before Ferrer and ...       T5   
 ..                          

# Run

In [31]:
relation_records = [] # null relation records
audits = [] # null audits records
additional_entities = [] # null additional entities records
# making empty table of additional entities
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])
tables['relations'] = pd.DataFrame(columns=['id','type','logic','certainty','entityIds','origin'])

logger.info(f"Start")
cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions', 'guglielmites_persons']) # , 'guglielmites_locations'
# cp = ParseContr oller(entity_list=['guglielmites_persons', 'guglielmites_locations'])
# cp = ParseController(entity_list=['guglielmites_events'])
cp.parse()
logger.info(f"End")

2022-09-22 15:02:55,325 INFO Start
2022-09-22 15:02:55,351 INFO TEXTS Uncertain parsing instructions in 0 columns: .
2022-09-22 15:02:55,363 INFO TEXTS 64 columns have been dropped (discard:64, unknown:0: []). Table now has 52 columns, inside:9,propvalue:15, special:6, proptype: 4, proptype_2nd: 0, dependent:4, reference_part:0. Originally 116 columns.
2022-09-22 15:02:55,378 INFO MANUSCRIPTS Uncertain parsing instructions in 0 columns: .
2022-09-22 15:02:55,384 INFO MANUSCRIPTS 35 columns have been dropped (discard:35, unknown:0: []). Table now has 31 columns, inside:8,propvalue:9, special:6, proptype: 4, proptype_2nd: 0, dependent:4, reference_part:0. Originally 66 columns.
2022-09-22 15:02:55,390 INFO RESOURCES Uncertain parsing instructions in 0 columns: .
2022-09-22 15:02:55,400 INFO RESOURCES 15 columns have been dropped (discard:15, unknown:0: []). Table now has 13 columns, inside:7,propvalue:3, special:1, proptype: 0, proptype_2nd: 0, dependent:0, reference_part:0. Originally 2

In [ ]:
tables['relations']

In [ ]:
json.dumps(relation_records)

### Profiling

In [ ]:
#%reload_ext line_profiler

# def profile():
#    cp.parsers['concepts'].parse_rows()

#%lprun -f EntityMapper.get_entity_id profile()

In [ ]:
# %prun -s tottime profile()

In [ ]:
# %lprun?

In [ ]:
tables['events']

In [ ]:
# what is this d72b2bee-47b4-4abc-8c88-793c72ab676d ?

pd.DataFrame(additional_entities)

### Last checks

In [ ]:
print(json.dumps(cp.parsers['actions'].js_objects[0]))

In [ ]:
logger.info(f"There are {len(additional_entities)} additionally created entities (e.g. values, resources ...).")

# Output the parsed data

In [ ]:
# root data
root_territory = """
 {
    "id": "T0",
    "class": "T",
    "data": { "parent": false },
    "label": "root",
    "detail": "",
    "language": "lat",
    "notes": [],
    "props": []
  }
"""

In [ ]:
# connects outcomes from the individual parsers
cp.load_json_objects()

# save json object list of the parsed entities from google sheets in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(cp.js_objects, f)

# save json object list of the "newly created entities" in the text file
with open('../datasets/all-test/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(additional_entities, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(additional_entities)

# write new and combine with old test data
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))

# write just the new parse data to the entities json.
with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))


# write relations
with open('../datasets/all-parsed/relations.json','w', encoding='utf-8') as f:
    json.dump(relation_records, f)

# write the audits json.
with open('../datasets/all-parsed/audits.json','w', encoding='utf-8') as f:
    json.dump(audits, f)


logger.info("END")

In [ ]:
# Log tables google sheet
# https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE/edit#gid=0


# d.open_gsheet("logtables","https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE")

# d.write_df_to_gsheet("logtables", "values", tables["values"])
# d.write_df_to_gsheet("logtables", "resources", tables["resources"])
# d.write_df_to_gsheet("logtables", "locations", tables["locations"])
# d.write_df_to_gsheet("logtables", "events", tables["events"])
# d.write_df_to_gsheet("logtables", "props", tables["props"])

